# Topic 42: Tuning Neural Networks

1. Pretrained networks
2. Using GridSearch/Talos for finding optimal parameter combinations
3. Saving your neural network to disk
4. Recurrent Neural Networks

## 1. Pretrained networks

* A pretrained network (also known as a convolutional base for CNNs) consists of layers that have already been trained on typically general data
* For images, these layers have already learned general patterns, textures, colors, etc. such that when you feed in your training data, certain features can immediately be detected. This part is **feature extraction**.
* You typically add your own final layers to train the network to classify/regress based on your problem. This component is **fine tuning**

Here are the pretrained models that exist within Keras: https://keras.io/api/applications/

To demonstrate the utility of pretrained networks, we'll compare model performance between a baseline model and a model using a pretrained network (VGG19).

In [ ]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

from tensorflow.keras.applications import VGG19

### Adding Pretrained Layers

VGG19: https://keras.io/api/applications/vgg/#vgg19-function

In [ ]:
pretrained = VGG19(weights='imagenet', 
                 include_top=True, 
                 input_shape=(224, 224, 3))

In [ ]:
pretrained.summary()

In [ ]:
pretrained = VGG19(weights='imagenet', 
                 include_top=False, 
                 input_shape=(224, 224, 3))

pretrained.summary()

In [ ]:
cnn = Sequential()
cnn.add(pretrained)

# freezing layers so they don't get retrained with your new data
for layer in cnn.layers:
    layer.trainable=False 

In [ ]:
# adding our own dense layers
cnn.add(Flatten())
cnn.add(Dense(132, activation='relu'))
cnn.add(Dense(1, activation='softmax'))

In [ ]:
# to verify that the weights are "frozen" 
for layer in cnn.layers:
    print(layer.name, layer.trainable)

With this you can now compile and fit your model!

## 2. GridSearch with Keras

There are a couple ways to go about testing combinations of parameters, GridSearch style:
* Using GridSearch: https://chrisalbon.com/deep_learning/keras/tuning_neural_network_hyperparameters/
    * This involves creating a model object such that scikit-learn's existing GridSearch functions work with your neural net
* Using Talos: https://autonomio.github.io/talos/#/Scan
    * This library lets you tune without having to create the model object, and also can automatically output your parameter combination scores into a csv file

In [ ]:
import talos

import pandas as pd
import numpy
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv",header=None, delimiter=",")

# Specify the data 
X = dataset.iloc[:,0:8]
y = dataset.iloc[:,8]

# Split the data up in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
def dense_network(x_train, y_train, x_test, y_test, params):

    # we build the model exactly like we would normally do it
    model = Sequential()

    # hidden layers
    model.add(Dense(params['nodes1'], input_dim=8, activation=params['activation1']))
    model.add(layers.Dropout(params['dropout']))
    model.add(Dense(12, activation=params['activation2']))
    
    # output layer
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=params['optimizer'], metrics=['accuracy'])
    
    out = model.fit(x_train, y_train, 
                        validation_data=(x_test, y_test),
                        batch_size=50,
                        epochs=10,
                        verbose=0)

    return out, model

In [ ]:
params = {'dropout': [0.1, 0.3, 0.5], 
          'nodes1': [12, 20, 100],
          'optimizer': ['adam', 'sgd'], 
          'activation1': ['relu', 'tanh'], 
          'activation2': ['relu', 'tanh']}

In [ ]:
results = talos.Scan(X_train.values, y_train.values, params=params, model=dense_network, experiment_name='grid')


In [ ]:
results.best_model(metric='accuracy')

In [ ]:
pd.read_csv('grid/012021182905.csv').sort_values('val_accuracy', ascending=False)

Another Talos walkthrough: https://medium.com/swlh/how-to-perform-keras-hyperparameter-optimization-x3-faster-on-tpu-for-free-602b97812602

## 3. Saving your model

In [ ]:
model = Sequential()

# hidden layers
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(Dense(12, activation='relu'))

# output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                    batch_size=50,
                    epochs=200,
                    verbose=1)

In [ ]:
import matplotlib.pyplot as plt

# Get training and test loss/accuracy histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

training_acc = history.history['accuracy']
test_acc = history.history['val_accuracy']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# Visualize loss history
ax1.plot(epoch_count, training_loss, 'r--')
ax1.plot(epoch_count, test_loss, 'b-')
ax1.legend(['Training Loss', 'Test Loss'])

# Visualize accuracy  history
ax2.plot(epoch_count, training_acc, 'r--')
ax2.plot(epoch_count, test_acc, 'b-')
ax2.legend(['Training Accuracy', 'Test Accuracy'])

fig.show();

In [ ]:
model.save('model.h5')
model.save_weights('model_weights.h5')

In [ ]:
from keras.models import load_model

my_model = load_model('model.h5')
my_model.load_weights('model_weights.h5')

In [ ]:
my_model.evaluate(X_test.values, y_test.values)

## 4. Recurrent Neural Networks - A Quick Overview

Some examples:
* https://towardsdatascience.com/recurrent-neural-networks-by-example-in-python-ffd204f99470
* https://towardsdatascience.com/a-practical-guide-to-rnn-and-lstm-in-keras-980f176271bc
* https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/